In [ ]:
import numpy as np
import PIL
import time

In [ ]:
'''
#just some little tests

a = time.time()
img = Image.open("imgs/IMG_0002.PNG").convert("RGBA")
b = time.time()
print("load and convert:", b - a)
a = time.time()
img2 = Image.open("imgs/IMG_0042.jpg").convert("RGBA").resize((275,192)).rotate(-2, expand=1)
b = time.time()
print("load, convert, resize, rotate:", b - a)

a = time.time()
img.paste(img2, (245,45), mask=img2)
b = time.time()
print("paste:", b - a)

a = time.time()
img = img.filter(ImageFilter.GaussianBlur(5))
b = time.time()
print("blur:", b - a)

a = time.time()
img = img.resize((275,192), resample=Image.Resampling.LANCZOS)
b = time.time()
print("lanczos:", b - a)

img.show()
'''

In [ ]:
def mse(a, b):
    a = np.array(a)
    b = np.array(b)
    return np.mean((a - b) ** 2)

def optimizer(goal, stamp):
    goalSize = goal.size
    bestPos = goalSize / 2
    bestRot = 0
    absGradient = 100
    while (absGradient > 0.1):
        #oh god, what now? How to do the scales?
    return bestPos, bestRot